In [2]:
import math
import pandas as pd
from numpy import *
from matplotlib.pyplot import * 
from mpl_toolkits.axes_grid1 import ImageGrid
from scipy.stats import skew

rc('text', usetex=True, fontsize=20)

In [3]:
num_thetas = 9
cos_thetas = linspace(0,1,num_thetas+1) 
cos_thetas_plus = linspace(-1,0,num_thetas+1) 

thetas = sort(array([arccos(ct) for ct in cos_thetas]))
thetas_plus = array([arccos(ctp) for ctp in cos_thetas_plus])

In [4]:
print thetas
print thetas_plus
print rad2deg(thetas)
print rad2deg(thetas_plus)

[ 0.          0.47588225  0.67967382  0.84106867  0.98176536  1.11024234
  1.23095942  1.34670323  1.45945531  1.57079633]
[ 3.14159265  2.6657104   2.46191883  2.30052398  2.1598273   2.03135032
  1.91063324  1.79488942  1.68213734  1.57079633]
[  0.          27.26604445  38.94244127  48.1896851   56.2510114
  63.61220004  70.52877937  77.16041159  83.62062979  90.        ]
[ 180.          152.73395555  141.05755873  131.8103149   123.7489886
  116.38779996  109.47122063  102.83958841   96.37937021   90.        ]


In [5]:
### --------------- tau10E5 --------------- ###

vrots = [0]
vouts = [5,25,50,100,200]

all_x_frec = []
all_z_u = []
vrot_labs = []
vout_labs = []

for vout in vouts:
    for vrot in vrots:
        df = pd.read_csv('../../data/tau10E5/vrot'+str(vrot)+'/vout'+str(vout)+'/tau10E5_vrot'+str(vrot)+'_vout'+str(vout)+'_out.ascii', delimiter=' ')
        
        escaped = df['escaped']
        x_frec = df['x_frec']
        z_u = df['z_u']
        
        inds = where(escaped == 0)[0]
        
        x_frec_escaped = array(x_frec)[inds]
        z_u_escaped = array(z_u)[inds]
        
        all_x_frec.append(x_frec_escaped)
        all_z_u.append(z_u_escaped)
              
        vrot_labs.append(vrot)
        vout_labs.append(vout)

In [29]:
info_file = open('info_tau10E5.dat','w') 
info_file.write('vrot\tvout\ttheta_lower\ttheta_upper\tstd\tskw\n') # if skw > 0 there is more weight in the left tail of the distribution

for t in range(num_thetas):

    fig = figure(1, (12., 48.))
    grid = ImageGrid(fig, 111, # similar to subplot(111)
                    nrows_ncols = (5, 1), # creates 5x1 grid of axes
                    axes_pad=0.3, # pad between axes in inch.
                    aspect=False,)

    props = dict(boxstyle='square', facecolor='white')

    theta_lower = thetas[t]
    theta_upper = thetas[t+1]
    
    theta_plus_lower = thetas_plus[t+1]
    theta_plus_upper = thetas_plus[t]
    
    print("theta",theta_lower)
    
    for i in range(5):
        
        current_z_u = all_z_u[i]
        current_x_frec = all_x_frec[i]
        
        acos_current_z_u = array([arccos(czu) for czu in current_z_u])
        
        angle_indices = where( ((acos_current_z_u >= theta_lower) & (acos_current_z_u < theta_upper)) | ((acos_current_z_u >= theta_plus_lower) & (acos_current_z_u < theta_plus_upper)))[0]
        
        current_x = current_x_frec[angle_indices] #only the ones between those upper and lower angles
        
        vrot_lab = r'${0:.0f}$'.format(vrot_labs[i])
        vout_lab = r'${0:.0f}$'.format(vout_labs[i])
        theta_lower_lab = r'${0:.0f}$'.format(int(rad2deg(theta_lower)))
        theta_upper_lab = r'${0:.0f}$'.format(int(rad2deg(theta_upper)))

        theta_lab = r'$\theta=$ '
        lab = '$v_{rot}=$ '+vrot_lab+' $\mathrm{km}$ $\mathrm{s^{-1}}$ \n $v_{out}=$ '+vout_lab+' $\mathrm{km}$ $\mathrm{s^{-1}}$ \n'+theta_lab+theta_lower_lab+'$^\circ$ $-$ '+theta_upper_lab+'$^\circ$'
        
        grid_i = grid[i]

        n,b,p = grid_i.hist(current_x, bins=30, histtype='stepfilled', color='c')
        
        print(sum(n))

        grid_i.axvline(x=0, ymin=0, ymax=2200, c='k', linestyle='--')
        grid_i.set_xlim(-50,20)
        grid_i.set_ylim(0,2200)
        grid_i.set_ylabel('$\mathrm{Intensity}$')
        grid_i.set_xlabel('$\mathrm{x}$')
        grid_i.text(-45,1750, lab, fontsize=20, bbox=props)
        
        info_file.write(str(vrot_labs[i])+'\t'+str(vout_labs[i])+'\t'+str(theta_lower)+'\t'+str(theta_upper)+'\t'+str(std(current_x))+'\t'+str(skew(current_x))+'\n')

    savefig('./tau10E5/tau10E5_phi'+str(int(rad2deg(theta_lower)))+'-'+str(int(rad2deg(theta_upper)))+'.png', format='png', transparent=False, bbox_inches='tight')
    close()
    
info_file.close()

('theta', 0.0)
11215.0
11256.0
10984.0
11119.0
11032.0
('theta', 0.47588224966041665)
10953.0
11192.0
11175.0
11104.0
11090.0
('theta', 0.67967381890824408)
11124.0
11075.0
11203.0
11181.0
11350.0
('theta', 0.84106867056793033)
11083.0
10977.0
11090.0
11020.0
10996.0
('theta', 0.98176535657862274)
11356.0
11015.0
11029.0
11071.0
11017.0
('theta', 1.1102423351135742)
10902.0
11069.0
11123.0
11045.0
11084.0
('theta', 1.2309594173407747)
11114.0
11209.0
11221.0
11126.0
11106.0
('theta', 1.3467032344935257)
11138.0
11034.0
10966.0
11010.0
11294.0
('theta', 1.4594553124539327)
11111.0
11169.0
11205.0
11320.0
11027.0


In [32]:
### --------------- tau10E6 --------------- ###

vrots = [0]
vouts = [5,25,50,100,200]

all_x_frec = []
all_z_u = []
vrot_labs = []
vout_labs = []

for vout in vouts:
    for vrot in vrots:
        df = pd.read_csv('../../data/tau10E6/vrot'+str(vrot)+'/vout'+str(vout)+'/tau10E6_vrot'+str(vrot)+'_vout'+str(vout)+'_out.ascii', delimiter=' ')
        
        escaped = df['escaped']
        x_frec = df['x_frec']
        z_u = df['z_u']
        
        inds = where(escaped == 0)[0]
        
        x_frec_escaped = array(x_frec)[inds]
        z_u_escaped = array(z_u)[inds]
        
        all_x_frec.append(x_frec_escaped)
        all_z_u.append(z_u_escaped)
              
        vrot_labs.append(vrot)
        vout_labs.append(vout)

In [33]:
info_file = open('info_tau10E6.dat','w') 
info_file.write('vrot\tvout\ttheta_lower\ttheta_upper\tstd\tskw\n') # if skw > 0 there is more weight in the left tail of the distribution

for t in range(num_thetas):

    fig = figure(1, (12., 48.))
    grid = ImageGrid(fig, 111, # similar to subplot(111)
                    nrows_ncols = (5, 1), # creates 5x1 grid of axes
                    axes_pad=0.3, # pad between axes in inch.
                    aspect=False,)

    props = dict(boxstyle='square', facecolor='white')

    theta_lower = thetas[t]
    theta_upper = thetas[t+1]
    
    theta_plus_lower = thetas_plus[t+1]
    theta_plus_upper = thetas_plus[t]
    
    for i in range(5):
        
        current_z_u = all_z_u[i]
        current_x_frec = all_x_frec[i]
        
        acos_current_z_u = array([arccos(czu) for czu in current_z_u])
        
        angle_indices = where( ((acos_current_z_u >= theta_lower) & (acos_current_z_u < theta_upper)) | ((acos_current_z_u >= theta_plus_lower) & (acos_current_z_u < theta_plus_upper)))[0]
        
        current_x = current_x_frec[angle_indices] #only the ones between those upper and lower angles
        
        vrot_lab = r'${0:.0f}$'.format(vrot_labs[i])
        vout_lab = r'${0:.0f}$'.format(vout_labs[i])
        theta_lower_lab = r'${0:.0f}$'.format(int(rad2deg(theta_lower)))
        theta_upper_lab = r'${0:.0f}$'.format(int(rad2deg(theta_upper)))

        theta_lab = r'$\theta=$ '
        lab = '$v_{rot}=$ '+vrot_lab+' $\mathrm{km}$ $\mathrm{s^{-1}}$ \n $v_{out}=$ '+vout_lab+' $\mathrm{km}$ $\mathrm{s^{-1}}$ \n'+theta_lab+theta_lower_lab+'$^\circ$ $-$ '+theta_upper_lab+'$^\circ$'
        
        grid_i = grid[i]

        n,b,p = grid_i.hist(current_x, bins=30, histtype='stepfilled', color='c')

        grid_i.axvline(x=0, ymin=0, ymax=2200, c='k', linestyle='--')
        grid_i.set_xlim(-50,20)
        grid_i.set_ylim(0,2200)
        grid_i.set_ylabel('$\mathrm{Intensity}$')
        grid_i.set_xlabel('$\mathrm{x}$')
        grid_i.text(-45,1750, lab, fontsize=20, bbox=props)
        
        info_file.write(str(vrot_labs[i])+'\t'+str(vout_labs[i])+'\t'+str(theta_lower)+'\t'+str(theta_upper)+'\t'+str(std(current_x))+'\t'+str(skew(current_x))+'\n')

    savefig('./tau10E6/tau10E6_phi'+str(int(rad2deg(theta_lower)))+'-'+str(int(rad2deg(theta_upper)))+'.png', format='png', transparent=False, bbox_inches='tight')
    close()
    
info_file.close()

In [31]:
### --------------- tau10E7 --------------- ###

vrots = [0]
vouts = [5,25,50,100,200]

all_x_frec = []
all_z_u = []
vrot_labs = []
vout_labs = []

for vout in vouts:
    for vrot in vrots:
        df = pd.read_csv('../../data/tau10E7/vrot'+str(vrot)+'/vout'+str(vout)+'/tau10E7_vrot'+str(vrot)+'_vout'+str(vout)+'_out.ascii', delimiter=' ')
        
        escaped = df['escaped']
        x_frec = df['x_frec']
        z_u = df['z_u']
        
        inds = where(escaped == 0)[0]
        
        x_frec_escaped = array(x_frec)[inds]
        z_u_escaped = array(z_u)[inds]
        
        all_x_frec.append(x_frec_escaped)
        all_z_u.append(z_u_escaped)
              
        vrot_labs.append(vrot)
        vout_labs.append(vout)

In [34]:
info_file = open('info_tau10E7.dat','w') 
info_file.write('vrot\tvout\ttheta_lower\ttheta_upper\tstd\tskw\n') # if skw > 0 there is more weight in the left tail of the distribution

for t in range(num_thetas):

    fig = figure(1, (12., 48.))
    grid = ImageGrid(fig, 111, # similar to subplot(111)
                    nrows_ncols = (5, 1), # creates 5x1 grid of axes
                    axes_pad=0.3, # pad between axes in inch.
                    aspect=False,)

    props = dict(boxstyle='square', facecolor='white')

    theta_lower = thetas[t]
    theta_upper = thetas[t+1]
    
    theta_plus_lower = thetas_plus[t+1]
    theta_plus_upper = thetas_plus[t]
    
    for i in range(5):
        
        current_z_u = all_z_u[i]
        current_x_frec = all_x_frec[i]
        
        acos_current_z_u = array([arccos(czu) for czu in current_z_u])
        
        angle_indices = where( ((acos_current_z_u >= theta_lower) & (acos_current_z_u < theta_upper)) | ((acos_current_z_u >= theta_plus_lower) & (acos_current_z_u < theta_plus_upper)))[0]
        
        current_x = current_x_frec[angle_indices] #only the ones between those upper and lower angles
        
        vrot_lab = r'${0:.0f}$'.format(vrot_labs[i])
        vout_lab = r'${0:.0f}$'.format(vout_labs[i])
        theta_lower_lab = r'${0:.0f}$'.format(int(rad2deg(theta_lower)))
        theta_upper_lab = r'${0:.0f}$'.format(int(rad2deg(theta_upper)))

        theta_lab = r'$\theta=$ '
        lab = '$v_{rot}=$ '+vrot_lab+' $\mathrm{km}$ $\mathrm{s^{-1}}$ \n $v_{out}=$ '+vout_lab+' $\mathrm{km}$ $\mathrm{s^{-1}}$ \n'+theta_lab+theta_lower_lab+'$^\circ$ $-$ '+theta_upper_lab+'$^\circ$'
        
        grid_i = grid[i]

        n,b,p = grid_i.hist(current_x, bins=30, histtype='stepfilled', color='c')

        grid_i.axvline(x=0, ymin=0, ymax=2200, c='k', linestyle='--')
        grid_i.set_xlim(-50,20)
        grid_i.set_ylim(0,2200)
        grid_i.set_ylabel('$\mathrm{Intensity}$')
        grid_i.set_xlabel('$\mathrm{x}$')
        grid_i.text(-45,1750, lab, fontsize=20, bbox=props)
        
        info_file.write(str(vrot_labs[i])+'\t'+str(vout_labs[i])+'\t'+str(theta_lower)+'\t'+str(theta_upper)+'\t'+str(std(current_x))+'\t'+str(skew(current_x))+'\n')

    savefig('./tau10E7/tau10E7_phi'+str(int(rad2deg(theta_lower)))+'-'+str(int(rad2deg(theta_upper)))+'.png', format='png', transparent=False, bbox_inches='tight')
    close()
    
info_file.close()